In [1]:
import  os
import  pandas as pd
from    tqdm import tqdm
import  numpy as np
from    multiprocessing import get_context as context
from    matplotlib import pyplot as plt
%matplotlib inline

# Read the files

# Parameters


header      = ['EventID','TrackID','Particle','EnergyDeposited','XPosition','YPosition','ZPosition','LocalTime','Volume', 'Initial Energy', 'Origin Volume', 'MigrantID']
PROCESSES   = os.cpu_count()-1
Pool        = context("fork").Pool

# Find the files
foldername  = f'../build/output'
os.system(f'rm -rf ../build/output/.ipynb_checkpoints')
filenames   = os.listdir(foldername)

print(f'Found {len(filenames)} data files in {foldername}')

def parallel(function):
    def inner(input_array):
        # Parallelize excecution them
        with Pool(PROCESSES) as pool:
            output = list(tqdm(pool.imap(function, input_array),total=len(input_array)))
        return output 
    return inner

# Function to load a single file
def loadfile(filename):
    df = pd.read_csv(os.path.join(foldername,filename),skiprows=16,names=header)
    df.insert(0, 'Filename', filename)
    return df

Found 19 data files in ../build/output


In [2]:
def processFile(filename):
    
    veto = loadfile(filename)
    veto = veto[veto.Volume.str.contains('veto')]
    veto = veto.groupby(['Filename','EventID', 'Volume'])['EnergyDeposited'].sum().reset_index()
    veto = veto[veto.EnergyDeposited > 10**(-9)]


    # loading the rest of the dataframe
    data = loadfile(filename)
    data = data[~(data.Volume.str.contains('veto'))]
    data = data.groupby(['Filename','EventID', 'Volume'])['EnergyDeposited'].sum().reset_index() # sum the energy per volume, hit and eventid are overlapping perfectly
    data = data[data['EnergyDeposited'] > 10**(-9)]

    return veto,data


processFiles        = parallel(processFile)
processed           = processFiles(filenames)

100%|██████████| 19/19 [00:09<00:00,  2.03it/s]


In [3]:
vetos = [i[0] for i in processed]
datas = [i[1] for i in processed]

In [4]:
veto = pd.concat(vetos)
data = pd.concat(datas)

In [5]:
veto = veto.rename(columns={'EnergyDeposited':'vetoEnergy'})
veto = veto.drop(columns='Volume')

In [6]:
crystal1 = data[data['Volume'] == 'crystal1_PV']
crystal1_2 = data[data['Volume'].isin(['crystal1_PV','crystal2_PV'])]
crystal1_2_3_4 = data[data['Volume'].isin(['crystal1_PV','crystal2_PV','crystal3_PV','crystal4_PV'])]

In [7]:
# Veto + crystal1 coincidence

def coincidence(data1: pd.DataFrame, data2: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    This function takes in dataframes that have only one volume.
    It merges them based on their EventID and sums their energies
    """
    combined = pd.DataFrame.merge(data1, data2, how = 'inner', on=['Filename','EventID'])
    if 'EnergyDeposited_x' in combined.columns:
        combined['EnergyDeposited'] = combined['EnergyDeposited_x'] + combined['EnergyDeposited_y']
    columns_to_drop = ['EnergyDeposited_x','EnergyDeposited_y', 'Volume_x', 'Volume_y','Volume']
    for column in columns_to_drop:
        if column in combined.columns:
            combined = combined.drop(columns=column)
    return combined

In [8]:
veto_and_1 = coincidence(veto,crystal1)


In [9]:
veto_and_1

,Filename,EventID,vetoEnergy,EnergyDeposited
0,TES-run0-rank8_nt_hits.csv,1734,1.514370,10.390381
1,TES-run0-rank8_nt_hits.csv,11150,2.928880,9.772884
2,TES-run0-rank8_nt_hits.csv,12117,1.394302,13.840228
3,TES-run0-rank8_nt_hits.csv,12895,0.371611,22.475888
4,TES-run0-rank8_nt_hits.csv,16526,1.875390,4.911784
...,...,...,...,...
39351,TES-run0-rank4_nt_hits.csv,5249848,3.568518,11.353800
39352,TES-run0-rank4_nt_hits.csv,5250302,3.637194,20.850754
39353,TES-run0-rank4_nt_hits.csv,5252160,1.475250,14.580829
39354,TES-run0-rank4_nt_hits.csv,5256075,1.825100,1.789590


In [10]:
veto_and_1or2 = coincidence(veto,crystal1_2)
veto_and_1or2

,Filename,EventID,vetoEnergy,EnergyDeposited
0,TES-run0-rank8_nt_hits.csv,50,7.287535,14.721072
1,TES-run0-rank8_nt_hits.csv,1734,1.514370,10.390381
2,TES-run0-rank8_nt_hits.csv,4293,1.464930,15.228317
3,TES-run0-rank8_nt_hits.csv,6584,0.051097,0.793989
4,TES-run0-rank8_nt_hits.csv,7969,3.706150,11.377783
...,...,...,...,...
79049,TES-run0-rank4_nt_hits.csv,5258704,1.517810,9.778362
79050,TES-run0-rank4_nt_hits.csv,5260109,3.141085,8.649230
79051,TES-run0-rank4_nt_hits.csv,5261092,2.129720,7.113230
79052,TES-run0-rank4_nt_hits.csv,5261323,4.383765,28.643256


In [11]:
veto_and_1or2or3or4 = coincidence(veto,crystal1_2_3_4)
veto_and_1or2or3or4

,Filename,EventID,vetoEnergy,EnergyDeposited
0,TES-run0-rank8_nt_hits.csv,50,7.287535,14.721072
1,TES-run0-rank8_nt_hits.csv,216,6.064633,20.132300
2,TES-run0-rank8_nt_hits.csv,717,2.434812,6.270798
3,TES-run0-rank8_nt_hits.csv,1524,5.133815,38.188515
4,TES-run0-rank8_nt_hits.csv,1734,1.514370,10.390381
...,...,...,...,...
158264,TES-run0-rank4_nt_hits.csv,5261092,2.129720,7.113230
158265,TES-run0-rank4_nt_hits.csv,5261113,1.951943,4.007384
158266,TES-run0-rank4_nt_hits.csv,5261323,4.383765,28.643256
158267,TES-run0-rank4_nt_hits.csv,5261665,2.079850,0.464072


In [12]:
print(len(veto_and_1)/len(veto))

0.11398021361877621


In [13]:
print(len(veto_and_1or2)/len(veto))

0.22895090475197516


In [14]:
print(len(veto_and_1or2or3or4)/len(veto))

0.4583680869303306
